In [90]:
import numpy as np
import pandas as pd
from scipy.stats import norm, chi2
import sys

In [ ]:
# #preallocate image matrices for choices
# #This pertains to estimating covariance matrices of the error differences
# #See Train book on discrete choice analysis p 113
# #"This matrix can be used to transform the covariance matrix of
# #errors into the covariance matrix of error differences: ~Ωi = MiΩMi.T .
# temp = np.identity(Jm-1)
# M = np.empty((Jm, Jm-1, 12))
# for i in range(1, Jm+1):
#     M[i-1] = np.concatenate((temp[:,0:i-1], -1*np.ones((Jm-1,1)), temp[:, i-1:]), axis=1)

# #Matrices for only the chosen options
# Mi=M[y-1]



In [2]:
def DivisiveNormalization(theta, data):
    denom = theta[0] + np.multiply(theta[1], np.linalg.norm(data, theta[2], 1))
    v=np.divide(data.T, denom)
    
    return v

In [3]:
def calcPiProbitQuad(Mi, v):
    
    MiT=np.transpose(Mi, axes=(0,2,1))
    T=v.shape[0]
    [x, w] = np.polynomial.hermite.hermgauss(100)

    #I honestly don't really know how tensordot works, but these lines of code return the correct values
    c = np.tensordot(MiT,v, axes=([1,0]))
    cT=np.transpose(c, axes=(0,2,1))
    vi = cT.diagonal() #This matches vi in MATLAB for s=1, trials 8,10,14
    
    #first part of equation in ProbaChoice.m, line 242
    z1=np.multiply(-2**0.5, vi)

    #second part of equation in ProbaChoice.m, line 242
    z2=np.multiply(-2**0.5, x)

    #These values have been validated
    zz = [z1-ele for ele in z2]

    aa=np.prod(norm.cdf(zz), axis=1)
    #Pi have been validated
    Pi=np.divide(np.sum(np.multiply(w.reshape(100,1), aa), axis=0), np.pi**0.5)
    
    return Pi
    

In [25]:
def calcPiAll(theta, data):
    
    v=DivisiveNormalization(theta, data)
    
    probs = np.empty(data.shape)
    #get the size of the choice array. Choice arrays must be the same size
    Jm=data.shape[1]
    temp = np.identity(Jm-1)
    M = np.empty((Jm, Jm-1, Jm))


    for i in range(Jm):
        M[i] = np.concatenate((temp[:,0:i], -1*np.ones((Jm-1,1)), temp[:, i:]), axis=1)
    
    for i in range(Jm):
        y=np.array([i]*data.shape[0])
        #print(y)
        
        #Matrices for only the chosen options
        Mi=M[y]
        #print(Mi)
        
        pi=calcPiProbitQuad(Mi,v)
        probs[:,i]=pi.T
        #print("pi is {}".format(pi))
    return probs

In [33]:
# ###CONFIRMATORY ANALYSIS TO TEST MATCH WITH WEBB DATA
# #create a vector of the WTP values. These values are from data(1).X, cells 8, 10, and 14
# d = np.array([
#              [4, 2.33, 1.875, 1.8, 1.5, 1.495, 1.335, 1.275, 1.125, 1.09, 1, 0.925],
#              [2.125, 2.125, 2.025, 2.0, 1.875, 1.495, 1.485, 1.335, 1.275, 1.075, 1.0, 0.625],
#              [4.0, 2.17,  2.0, 2.0, 1.875, 1.875, 1.5, 1.485, 1.335, 1.275, 1.09, 1.075],
#              ])

# # #These are the chosen options for s=1, on trials 8, 10,14
# y=np.array([4,3,2])

# # #Choice set size for trials 8,10,14 for subject 1
# Jm=12

# # #sigma, omega(w), beta
# theta = [0.0000, 0.2376, 0.9739]
# temp = np.identity(Jm-1)
# M = np.empty((Jm, Jm-1, 12))
# for i in range(1, Jm+1):
#     M[i-1] = np.concatenate((temp[:,0:i-1], -1*np.ones((Jm-1,1)), temp[:, i-1:]), axis=1)

# #Matrices for only the chosen options
# Mi=M[y-1]

# #This result has been spot checked against the values returned by the MATLAB code for data(1).X, cells 8, 10, and 14 
# v=DivisiveNormalization(theta=theta, data=d)
# pi=calcPiProbitQuad(Mi,v)

# print("probs for chosen options only: {}".format(pi))
# print(pi) 
# #[0.08327671 0.10499576 0.09305649]

# probs=calcPiAll(theta=theta,data=d)
# print("probs for all options: {}".format(probs))

probs for chosen options only: [0.08327671 0.10499576 0.09305649]
[0.08327671 0.10499576 0.09305649]
probs for all only: [[0.22598222 0.10790416 0.08645194 0.08327671 0.07151711 0.0713334
  0.06565703 0.06362727 0.05877966 0.05769393 0.05497855 0.05279801]
 [0.11051148 0.11051148 0.10499576 0.10365229 0.09714261 0.07936819
  0.07893925 0.07273064 0.0703627  0.06292494 0.06030962 0.04855103]
 [0.19882952 0.09305649 0.08619875 0.08619875 0.08142488 0.08142488
  0.06838878 0.06790556 0.06322691 0.06143192 0.0561619  0.05575166]]


# Simulation of user choices
We simulate 100,000 trials of each of the 3 choice sets and use the values yielded by the `DivisiveNormalization` method + a random noise vector and check that the choice probabilities are roughly in line with the analytic probabilities from `calcPiProbitQuad`.

In [ ]:
#def chose_item_dn(d, num_it=1000, theta = [0.0000, 0.2376, 0.9739])
freq_chosen = np.array([0., 0., 0.])
num_it = 100000
v = DivisiveNormalization(theta=theta, data=d)
# the following covariance matrix has the structure
# [ 1    0.5    ...    0.5 ]
# [ 0.5    1    ...    0.5 ]
# [ 0.5   ...    1    0.5  ]
# [ 0.5   0.5   ...    1   ]
cov = np.ones((12, 12)) * 0.5
cov[np.arange(12), np.arange(12)] = 1
mean = np.zeros(12)
for i in range(num_it):
    eps = np.random.multivariate_normal(mean, cov, size=3).T
    u = v + eps
    item_chosen = (u.argmax(axis=0) == (y-1)).astype(float)
    freq_chosen += item_chosen / num_it
    
print(freq_chosen)

###Steps to Computing a Power Analysis Given an Experimental Design and value of theta
1. Read in scores into correct np array format
2. Chose the item given its normalized value 
3. Calculate the probability of the chosen item

In [35]:
#Load data from Bollen et al., 2010
choice = pd.read_csv('/Users/amywinecoff/Documents/CITP/Research/Github/AgentChoiceSim/co1_wide.csv')  

#for now, remove the conditions with 5 options so I can figure out the code for a fixed set size
choice = choice[~choice['condition'].isin(['Top5', 'Top5_NR'])]

score_cols = [c for c in choice.columns if 'score' in c]
choice_set_vals = choice[score_cols]
choice_set_vals.head()

,score1,score2,score3,score4,score5,score6,score7,score8,score9,score10,score11,score12,score13,score14,score15,score16,score17,score18,score19,score20
0,39,38,38,38,38,38.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0
1,42,42,42,42,41,38.0,37.0,36.0,35.0,35.0,34.0,33.0,33.0,33.0,32.0,32.0,31.0,31.0,30.0,30.0
2,41,40,40,40,40,39.0,39.0,38.0,38.0,38.0,37.0,37.0,37.0,37.0,36.0,36.0,36.0,36.0,35.0,35.0
3,36,36,35,35,35,34.0,33.0,33.0,33.0,33.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,31.0
5,41,40,40,40,39,39.0,39.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0


In [75]:
d = choice_set_vals.values
#sigma, omega(w), beta
#theta_h1 = [0.0000, 0.2376, 0.9739]
probs=calcPiAll(theta=t, data=data)

In [71]:
#def chose_item_dn(d, theta = [0.0000, 0.2376, 0.9739]):
def chose_item(theta, data):
    probs=calcPiAll(theta=t, data=data)
    num_subj = data.shape[0]
    Jm = data.shape[1]

    v = DivisiveNormalization(theta=theta, data=data)
    # the following covariance matrix has the structure
    # [ 1    0.5    ...    0.5 ]
    # [ 0.5    1    ...    0.5 ]
    # [ 0.5   ...    1    0.5  ]
    # [ 0.5   0.5   ...    1   ]


    cov = np.ones((Jm, Jm)) * 0.5
    cov[np.arange(Jm), np.arange(Jm)] = 1
    mean = np.zeros(Jm)
    #for i in range(num_it):
    eps = np.random.multivariate_normal(mean, cov, size=num_subj).T
    u = v + eps
    item_chosen = u.argmax(axis=0)
    
    return item_chosen


In [207]:
cov = np.ones((Jm, Jm)) * 0.5
cov[np.arange(Jm), np.arange(Jm)] = 1
mean = np.zeros(Jm)
#for i in range(num_it):
eps = np.random.multivariate_normal(mean, cov, size=num_subj).T

eps

array([[-0.20258108,  0.79640852, -1.90983179, ..., -0.16620076,
        -0.32164325,  0.81209447],
       [ 0.06336897,  1.33351183, -1.18883466, ..., -1.42496442,
         0.8386626 , -0.32001687],
       [-0.59312559,  0.77977772, -1.21722388, ..., -0.11739122,
         0.67289077, -0.26231862],
       ...,
       [ 0.75840811,  0.7001978 , -0.43663535, ..., -0.2532854 ,
         1.39994878,  0.49489074],
       [ 1.03274762,  0.45018809, -0.72462905, ...,  0.53591083,
         1.68690702,  1.04620738],
       [ 0.8862977 ,  1.99041256,  0.94213761, ..., -2.97182834,
         0.06652529,  0.04122848]])

In [206]:
#cov = np.ones((Jm, Jm)) * 0.5
#cov[np.arange(Jm), np.arange(Jm)] = 1

#eps = np.random.multivariate_normal(mean, cov, size=num_subj).T
#eps.shape

In [189]:
#cov=np.matmul(L, L.T)

In [205]:
#M = np.eye(Jm, Jm)
#L = np.linalg.cholesky(np.matmul(np.matmul(M, np.eye(Jm)*.5), M.T)).T
#L

In [196]:

#np.eye(Jm)
#np.eye(Jm-1, Jm-1)

#M = np.concatenate((-1*np.ones((Jm,1)), np.eye(Jm, Jm)), 1)
#L_in = np.ones((Jm, Jm)) * 0.5
#L_in[np.arange(Jm), np.arange(Jm)] = 1
#covr
#mean = np.zeros(Jm-1)
#for i in range(num_it):
#eps = np.random.multivariate_normal(mean, L, size=num_subj).T
#cov = [[1, 0], [0, 100]]  # diagonal covariance

In [184]:
#L = np.linalg.cholesky(np.matmul(np.matmul(M, np.eye(Jm)*.5), M.T)).T

In [151]:
#Jm=20
#cov = np.ones((Jm, Jm)) * 0.5
#cov[np.arange(Jm), np.arange(Jm)] = 1

#cov

array([[1. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. , 0.5, 0.5, 0

In [137]:
def calcModelLL(theta, data, null_theta=None):
    """Calculates the log likelikihood given theta values for a DN model. If a null model is being tested,
    it will chose the item based on the alternative model, then calculate the probability of that choice, and the 
    log-likelihood given both the alternative model and the null model
    """
    
    probs=calcPiAll(theta=theta, data=data)
    item_chosen = chose_item(theta=theta, data=data) 
    chosen_probs=probs[np.arange(len(probs)), item_chosen]
    #add epsilon to all values to prevent divide by zero error
    chosen_probs = chosen_probs + sys.float_info.epsilon
    LL = sum(np.log(chosen_probs))
    
    if null_theta:
        null_probs=calcPiAll(theta=null_theta, data=data)
        #add epsilon to all values to prevent divide by zero error
        null_probs = null_probs + sys.float_info.epsilon

        
        null_chosen_probs = null_probs[np.arange(len(null_probs)), item_chosen]
        print("null chosen probs {}".format(null_chosen_probs))
        null_LL = sum(np.log(null_chosen_probs))
    else:
        null_LL = None
    
    return LL, null_LL, null_chosen_probs
    

In [138]:
d = np.array([
             [4, 2.33, 1.875, 1.8, 1.5, 1.495, 1.335, 1.275, 1.125, 1.09, 1, 0.925],              
             [2.125, 2.125, 2.025, 2.0, 1.875, 1.495, 1.485, 1.335, 1.275, 1.075, 1.0, 0.625],
             [4.0, 2.17,  2.0, 2.0, 1.875, 1.875, 1.5, 1.485, 1.335, 1.275, 1.09, 1.075],
            ])
#omega allowed to vary. Set to value in Webb et al., 2020
theta_h1 = [1.0, 0.117, 1.0]
#This is the null model that tests that omega != 0
theta_h0 = [theta_h1[0], 0, theta_h1[2]]

LLs = calcModelLL(theta=theta_h1, data=d, null_theta=theta_h0)
#LL_h1 = calcModelLL(theta=theta_h1, data=d)

print("LL for H0 model: {}".format(LLs[1]))
print("LL for H1 model: {}".format(LLs[0]))
#print(LL_h1)#-362.68216377703664
LR = 2*(LLs[0]-LLs[1])
print(LR)

null chosen probs [0.00362869 0.20155407 0.90836738]
LL for H0 model: -7.316688120269219
LL for H1 model: -6.123540503101526
2.3862952343353854


In [139]:
p=1 - chi2.cdf(LR, 1)
print(p)

0.12240340353995649


In [148]:
d = choice_set_vals.values /10
d

array([[3.9, 3.8, 3.8, ..., 3.7, 3.7, 3.7],
       [4.2, 4.2, 4.2, ..., 3.1, 3. , 3. ],
       [4.1, 4. , 4. , ..., 3.6, 3.5, 3.5],
       ...,
       [3.8, 3.8, 3.8, ..., 3.7, 3.7, 3.7],
       [4.7, 4.4, 4.3, ..., 3.9, 3.9, 3.8],
       [4. , 4. , 4. , ..., 3.8, 3.8, 3.8]])

In [210]:
d = choice_set_vals.values /20
#omega allowed to vary. Set to value in Webb et al., 2020
theta_h1 = [0.44, 0.0006, 1.0]
#This is the null model that tests that omega != 0
theta_h0 = [theta_h1[0], 0, theta_h1[2]]

LLs = calcModelLL(theta=theta_h1, data=d, null_theta=theta_h0)
#LL_h1 = calcModelLL(theta=theta_h1, data=d)

print("LL for H0 model: {}".format(LLs[1]))
print("LL for H1 model: {}".format(LLs[0]))
#print(LL_h1)#-362.68216377703664
LR = 2*(LLs[0]-LLs[1])
print(LR)
p=1 - chi2.cdf(LR, 1)
print(p)

null chosen probs [0.04407307 0.00939629 0.06456036 0.04393492 0.09855233 0.01955426
 0.0539972  0.04327555 0.03802913 0.16673013 0.18214653 0.11786076
 0.0464083  0.15902676 0.09713999 0.04428456 0.05321597 0.0517849
 0.04619155 0.05650952 0.03605358 0.0749311  0.07320281 0.04580393
 0.04102132 0.05707062 0.05348236 0.22143556 0.00544888 0.04911181
 0.05758466 0.03066397 0.11429435 0.11833017 0.06530469 0.03741784
 0.11166032 0.18699963 0.04407307 0.02594359 0.04049192 0.05449321
 0.030856   0.03786274 0.11626147 0.24295879 0.13647897 0.18371021
 0.06498833 0.00584738 0.1042198  0.04680746 0.0470266  0.06331734
 0.04173563 0.04244452 0.12513343 0.13486188 0.15964809 0.07081973
 0.04911181 0.06296969 0.06158799 0.1776817  0.04919191 0.16401814
 0.03826924 0.19991423 0.18486236 0.17713294 0.04636833 0.11376859
 0.05       0.09372814 0.06617543 0.1330733  0.08762272 0.16433096
 0.04892701 0.04277427 0.08009088 0.1592109  0.04830685 0.03605693
 0.04785476 0.02917572 0.14247729 0.12177073 

In [142]:
null_probs=calcPiAll(theta=theta_h0, data=d)

In [146]:
null_probs_df= pd.DataFrame(null_probs)
null_probs_df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.00,1.899591e-12,1.899591e-12,1.899591e-12,1.899591e-12,1.899591e-12,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45,4.629752e-45
1,0.25,2.500000e-01,2.500000e-01,2.500000e-01,9.059932e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,1.00,1.899634e-12,1.899634e-12,1.899634e-12,1.899634e-12,6.012044e-45,6.012044e-45,1.663142e-106,1.663142e-106,1.663142e-106,2.449656e-273,2.449656e-273,2.449656e-273,2.449656e-273,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,0.50,5.000000e-01,8.599785e-17,8.599785e-17,8.599785e-17,2.940557e-59,8.831573e-132,8.831573e-132,8.831573e-132,8.831573e-132,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,1.00,1.899677e-12,1.899677e-12,1.899677e-12,8.173178e-45,8.173178e-45,8.173178e-45,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105,2.681732e-105
5,0.25,2.500000e-01,2.500000e-01,2.500000e-01,9.059932e-21,2.512807e-181,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
6,0.50,5.000000e-01,8.599785e-17,8.599785e-17,8.599785e-17,8.599785e-17,8.599785e-17,8.599785e-17,8.599785e-17,8.599785e-17,8.599785e-17,8.599785e-17,2.906825e-59,2.906825e-59,2.906825e-59,2.906825e-59,2.906825e-59,2.906825e-59,2.906825e-59,2.906825e-59
7,1.00,1.899763e-12,1.909966e-44,1.909966e-44,1.909966e-44,1.909966e-44,1.909966e-44,1.909966e-44,3.716992e-101,3.716992e-101,3.716992e-101,3.716992e-101,3.716992e-101,3.716992e-101,3.716992e-101,1.261944e-199,1.261944e-199,1.261944e-199,1.261944e-199,1.261944e-199
8,1.00,1.899677e-12,1.899677e-12,1.899677e-12,8.173178e-45,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
9,0.25,2.500000e-01,2.500000e-01,2.500000e-01,9.059932e-21,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [147]:
choice_set_vals.head(10)

,score1,score2,score3,score4,score5,score6,score7,score8,score9,score10,score11,score12,score13,score14,score15,score16,score17,score18,score19,score20
0,39,38,38,38,38,38.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0,37.0
1,42,42,42,42,41,38.0,37.0,36.0,35.0,35.0,34.0,33.0,33.0,33.0,32.0,32.0,31.0,31.0,30.0,30.0
2,41,40,40,40,40,39.0,39.0,38.0,38.0,38.0,37.0,37.0,37.0,37.0,36.0,36.0,36.0,36.0,35.0,35.0
3,36,36,35,35,35,34.0,33.0,33.0,33.0,33.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,31.0
5,41,40,40,40,39,39.0,39.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0
6,39,39,39,39,38,36.0,35.0,35.0,34.0,34.0,34.0,34.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0
7,36,36,35,35,35,35.0,35.0,35.0,35.0,35.0,35.0,35.0,34.0,34.0,34.0,34.0,34.0,34.0,34.0,34.0
8,39,38,37,37,37,37.0,37.0,37.0,36.0,36.0,36.0,36.0,36.0,36.0,36.0,35.0,35.0,35.0,35.0,35.0
9,43,42,42,42,41,37.0,36.0,35.0,35.0,34.0,34.0,34.0,33.0,33.0,33.0,32.0,32.0,32.0,32.0,31.0
10,40,40,40,40,39,35.0,34.0,33.0,33.0,32.0,32.0,32.0,31.0,31.0,30.0,30.0,29.0,29.0,29.0,28.0


In [ ]:
# save as Python
#!jupyter nbconvert --to script DivisiveNormalization.ipynb